In [ ]:
"""
Created on 20190203
Reviewed on 20190203

@author: Ross.Lee

Ref: https://medium.com/@daniel820710/%E5%88%A9%E7%94%A8keras%E5%BB%BA%E6%A7%8Blstm%E6%A8%A1%E5%9E%8B-%E4%BB%A5stock-prediction-%E7%82%BA%E4%BE%8B-1-67456e0a0b
"""

# Import

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector, LeakyReLU, BatchNormalization, Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Date Preparation

In [4]:
def readTrain():
    train = pd.read_csv("SPY.csv")
    return train


def augFeatures(train):
    train["Date"] = pd.to_datetime(train["Date"])
    train["year"] = train["Date"].dt.year
    train["month"] = train["Date"].dt.month
    train["date"] = train["Date"].dt.day
    train["day"] = train["Date"].dt.dayofweek
    return train


def normalize(train):
    train = train.drop(["Date"], axis=1)
    train_norm = train.apply(lambda x: (
        x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm


def buildTrain(train, pastDay=30, futureDay=5):
    X_train, Y_train = [], []
    for i in range(train.shape[0] - futureDay - pastDay):
        # ex. 0-30 (idx 0~29), 1-31 (idx 1~30)
        X_train.append(np.array(train.iloc[i: i + pastDay]))
        Y_train.append(
            np.array(train.iloc[i + pastDay: i + pastDay + futureDay]["Adj Close"]))
    return np.array(X_train), np.array(Y_train)


def shuffle(X, Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]


def splitData(X, Y, rate):
    X_train = X[int(X.shape[0] * rate):]
    Y_train = Y[int(Y.shape[0] * rate):]
    X_val = X[:int(X.shape[0] * rate)]
    Y_val = Y[:int(Y.shape[0] * rate)]
    return X_train, Y_train, X_val, Y_val


if __name__ == "__main__":
        # read SPY.csv
    train = readTrain()

    # Augment the features (year, month, date, day)
    train_Aug = augFeatures(train)

    # Normalization
    train_norm = normalize(train_Aug)

    # build Data, use last 30 days to predict next 5 days
    X_train, Y_train = buildTrain(train_norm, 30, 5)

    # shuffle the data, and random seed is 10
    X_train, Y_train = shuffle(X_train, Y_train)

    # split training data and validation data
    X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
    # X_trian: (5710, 30, 10)
    # Y_train: (5710, 5, 1)
    # X_val: (634, 30, 10)
    # Y_val: (634, 5, 1)

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)

X_train shape (5680, 30, 10)
Y_train shape (5680, 5)


# Some Notes
 - RNN: (None, Timestep, Features)，None is the batch size
    - example 
        - one batch:  
                A  B  C  D  
              0 X  X  X  X  
              1 X  X  X  X  
              2 X  X  X  X  
              3 X  X  X  X  
        - 0~3 -> Timestep: RNN的每個sub-iter就會吃一列，全部跑完後再BackProp回來  
        - A~B -> Features: 每次RNN吃的特徵數目
 
 - One to one -> Timestep只有1，預測的y也是1筆。例如今天預測明天
 - many to one -> Timestep不只1，預測的y為1筆。例如前一個月預測明天
 - one to many -> Timestep只有1
 - many to many -> Timestep不只1，預測的y也不只1筆。例如前一個月預測未來5天



# Model Construction

## One to One RNN
`由於是一對一模型，因此return_sequences 也可設為False ，但Y_train 以及Y_val的維度需改為二維(5710,1)以及(634,1)，因為有return_sequences，所以會輸出timestep的維度。`

### Construct the model

In [5]:
def buildOneToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_shape=(shape[1], shape[2]), return_sequences=True))
    # output shape: (1, 1)
    model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

### Train the one to one model

In [7]:
train = readTrain()
train_Aug = augFeatures(train=train)
train_norm = normalize(train=train_Aug)
# change the last day and next day
X_train, Y_train = buildTrain(train=train_norm, pastDay=1, futureDay=1) # one to one
X_train, Y_train = shuffle(X=X_train, Y=Y_train)
X_train, Y_train, X_val, Y_val = splitData(X=X_train, Y=Y_train, rate=0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:, np.newaxis]
Y_val = Y_val[:, np.newaxis]

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)

# Keras callback is useful: https://keras.io/zh/callbacks/

model = buildOneToOneModel(X_train.shape)
earlystopping = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.001)
csvlog = CSVLogger('one2one_train_log')
model.fit(X_train, Y_train, epochs=1000, batch_size=128,
          validation_data=(X_val, Y_val), callbacks=[earlystopping, reducelr, csvlog])

X_train shape (5710, 1, 10)
Y_train shape (5710, 1, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1, 10)             840       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5710 samples, validate on 634 samples
Epoch 1/1000
5710/5710 [==============================] - 1s 123us/step - loss: 0.0719 - val_loss: 0.0531
Epoch 2/1000
5710/5710 [==============================] - 0s 13us/step - loss: 0.0424 - val_loss: 0.0289
Epoch 3/1000
5710/5710 [==============================] - 0s 13us/step - loss: 0.0202 - val_loss: 0.0114
Epoch 4/1000
5710/5710 [==============================] - 0s 13us/step - loss: 0.0065 - val_loss: 0.0027
Epoch 5/1000
5710

5710/5710 [==============================] - 0s 13us/step - loss: 2.4515e-05 - val_loss: 2.3676e-05
Epoch 139/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4261e-05 - val_loss: 2.3702e-05
Epoch 140/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.4301e-05 - val_loss: 2.3774e-05
Epoch 141/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4422e-05 - val_loss: 2.3602e-05
Epoch 142/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.4326e-05 - val_loss: 2.3895e-05
Epoch 143/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.3925e-05 - val_loss: 2.3473e-05
Epoch 144/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.3821e-05 - val_loss: 2.2932e-05
Epoch 145/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.3923e-05 - val_loss: 2.2983e-05
Epoch 146/1000
5710/5710 [==============================] - 0s 13us/step - loss: 2.3878e-05 - v

In [11]:
print(model.predict(X_train[0:10]))
print(Y_train[0:10])

[[[ 0.16894309]]

 [[-0.23473354]]

 [[-0.07928583]]

 [[-0.0411837 ]]

 [[ 0.00463182]]

 [[-0.2941407 ]]

 [[-0.02194393]]

 [[-0.03507504]]

 [[-0.00897813]]

 [[-0.02777452]]]
[[[ 1.70236981e-01]]

 [[-2.32875884e-01]]

 [[-8.32417025e-02]]

 [[-3.34503232e-02]]

 [[-1.48952916e-04]]

 [[-2.93853160e-01]]

 [[-1.93453977e-02]]

 [[-3.43362259e-02]]

 [[-2.59278393e-03]]

 [[-2.75129136e-02]]]


## Many ot one
`LSTM參數return_sequences=False ，未設定時default也為False，而且不可使用TimeDistribution`

### Construct the model

In [70]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_shape=(shape[1], shape[2])))
    # output shape: (1, 1)
    model.add(Dense(1))  # 不可使用TimeDistribution，否則輸出會變成(batch, 30, 10, 1)
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [62]:
# return_sequences=False && return_state=False 意義
# https://blog.csdn.net/u011327333/article/details/78501054
"""
1. return_sequences=False && return_state=False
    Keras API 中，return_sequences和return_state默認就是false。此時只會返回一個hidden state 值。
如果input 數據包含多個時間步，則這個hidden state 是最後一個時間步的結果

2. return_sequences=True && return_state=False
    e.g. LSTM(1, return_sequences=True)
    輸出的hidden state 包含全部時間步的結果。
    
3. return_sequences=True && return_state=True
    lstm1, state_h, state_c = LSTM(1, return_sequences=True, return_state=True)
    此時，我們既要輸出全部時間步的hidden state ，又要輸出cell state。
    lstm1 存放的就是全部時間步的hidden state。
    state_h 存放的是最後一個時間步的hidden state
    state_c 存放的是最後一個時間步的cell state

    一個輸出例子，假設我們輸入的時間步time step=3
    [array([[[-0.02145359],
        [-0.0540871 ],
        [-0.09228823]]], dtype=float32),  # lstm1
     array([[-0.09228823]], dtype=float32),  # state_h
     array([[-0.19803026]], dtype=float32)]  # state_c
     
ref: https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/
"""


model = Sequential()
model.add(LSTM(10, input_shape=(30, 10)))  # 吃最後一部hidden_state
# 使用TimeDistribution的return_seq要等於True，輸出會變成(batch, 30, 1) --> 吃每一步的hidden_state
# model.add(TimeDistributed(Dense(1)))
model.add(Dense(1)) 
model.compile(loss="mse", optimizer="adam")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 10)                840       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


### Train many to one model

In [67]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, pastDay=30, futureDay=1)  # many to one
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, rate=0.1)

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)

model = buildManyToOneModel(X_train.shape)
earlystopping = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.001)
csvlog = CSVLogger('many2one_train_log')
model.fit(X_train, Y_train, epochs=1000, batch_size=128,
          validation_data=(X_val, Y_val), callbacks=[earlystopping, reducelr, csvlog])

X_train shape (5684, 30, 10)
Y_train shape (5684, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 10)                840       
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5684 samples, validate on 631 samples
Epoch 1/1000
5684/5684 [==============================] - 2s 376us/step - loss: 0.1257 - val_loss: 0.0485
Epoch 2/1000
5684/5684 [==============================] - 1s 123us/step - loss: 0.0196 - val_loss: 0.0064
Epoch 3/1000
5684/5684 [==============================] - 1s 106us/step - loss: 0.0030 - val_loss: 0.0015
Epoch 4/1000
5684/5684 [==============================] - 1s 107us/step - loss: 0.0011 - val_loss: 8.5506e-04
Epoch 5/1000

5684/5684 [==============================] - 1s 111us/step - loss: 2.9774e-05 - val_loss: 3.2953e-05
Epoch 138/1000
5684/5684 [==============================] - 1s 114us/step - loss: 2.9559e-05 - val_loss: 3.2007e-05
Epoch 139/1000
5684/5684 [==============================] - 1s 110us/step - loss: 2.8904e-05 - val_loss: 3.6301e-05
Epoch 140/1000
5684/5684 [==============================] - 1s 120us/step - loss: 2.8463e-05 - val_loss: 3.4718e-05
Epoch 141/1000
5684/5684 [==============================] - 1s 105us/step - loss: 2.8344e-05 - val_loss: 3.0936e-05
Epoch 142/1000
5684/5684 [==============================] - 1s 109us/step - loss: 2.9023e-05 - val_loss: 3.2627e-05
Epoch 143/1000
5684/5684 [==============================] - 1s 106us/step - loss: 2.8559e-05 - val_loss: 3.0733e-05
Epoch 144/1000
5684/5684 [==============================] - 1s 108us/step - loss: 2.8509e-05 - val_loss: 3.4867e-05
Epoch 145/1000
5684/5684 [==============================] - 1s 116us/step - loss: 2.854

In [22]:
print(X_train[0:2].shape)
print(model.predict(X_train[0:2]))
print(Y_train[0:2])

(2, 30, 10)
[[ 0.03397147]
 [-0.083556  ]]
[[ 0.03222898]
 [-0.08687057]]


## One to many

### Construct the 
`因為是一對多模型Timesteps只有1，因此return_sequences=False 才可執行。只有return sequence的時候，才會回傳Timestep的維度`

In [68]:
def buildOneToManyModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_shape=(shape[1], shape[2])))
    # output shape: (5, 1)
    model.add(Dense(1))
    model.add(RepeatVector(5))  # 將輸出的向量x5，進行多步預測。所以y的dim要再多給1個:(None, 5, 1)。
    """
    RepeatVector:
        n :整數，重複次數。

        輸入尺寸:
           2D張量，尺寸為(num_samples, features)。

        輸出尺寸:
           3D張量，尺寸為(num_samples, n, features)。
    """
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [53]:
buildOneToManyModel((1, 5))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 10)                640       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 11        
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 5, 1)              0         
Total params: 651
Trainable params: 651
Non-trainable params: 0
_________________________________________________________________


### Train one to many model

In [72]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, pastDay=1, futureDay=5)  # many to one
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, rate=0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)

model = buildOneToManyModel(X_train.shape)
earlystopping = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.001)
csvlog = CSVLogger('one2many_train_log')
model.fit(X_train, Y_train, epochs=1000, batch_size=128,
          validation_data=(X_val, Y_val), callbacks=[earlystopping, reducelr, csvlog])

X_train shape (5706, 1, 10)
Y_train shape (5706, 5, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_29 (LSTM)               (None, 10)                840       
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 5, 1)              0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5706 samples, validate on 634 samples
Epoch 1/1000
5706/5706 [==============================] - 1s 260us/step - loss: 0.0719 - val_loss: 0.0552
Epoch 2/1000
5706/5706 [==============================] - 0s 13us/step - loss: 0.0423 - val_loss: 0.0295
Epoch 3/1000
5706/5706 [==============================] - 0s 14us/step - loss: 0.0202 - val_los

5706/5706 [==============================] - 0s 13us/step - loss: 7.2711e-05 - val_loss: 7.1611e-05
Epoch 67/1000
5706/5706 [==============================] - 0s 13us/step - loss: 7.2111e-05 - val_loss: 7.2394e-05
Epoch 68/1000
5706/5706 [==============================] - 0s 13us/step - loss: 7.1919e-05 - val_loss: 7.1000e-05
Epoch 69/1000
5706/5706 [==============================] - 0s 13us/step - loss: 7.1667e-05 - val_loss: 7.0433e-05
Epoch 70/1000
5706/5706 [==============================] - 0s 13us/step - loss: 7.1349e-05 - val_loss: 6.9456e-05
Epoch 71/1000
5706/5706 [==============================] - 0s 13us/step - loss: 7.0937e-05 - val_loss: 6.9533e-05
Epoch 72/1000
5706/5706 [==============================] - 0s 15us/step - loss: 7.0304e-05 - val_loss: 7.0569e-05
Epoch 73/1000
5706/5706 [==============================] - 0s 13us/step - loss: 7.0716e-05 - val_loss: 6.9059e-05
Epoch 74/1000
5706/5706 [==============================] - 0s 17us/step - loss: 7.0329e-05 - val_loss:

In [76]:
print(X_train[0:1].shape)
print(Y_train[0:1])
#
model.predict(X_train[0:1])

(1, 1, 10)
[[[0.16298736]
  [0.17023698]
  [0.17023698]
  [0.16428321]
  [0.15955536]]]


array([[[0.16140072],
        [0.16140072],
        [0.16140072],
        [0.16140072],
        [0.16140072]]], dtype=float32)

## Many to many
`(輸入與輸出相同長度) 將return_sequences 設為True ，再用TimeDistributed(Dense(1)) 將輸出調整為(5,1)`

### Construct the model

In [78]:
def buildManyToManyModel(shape):
    model = Sequential()
    model.add(
        LSTM(10, input_shape=(shape[1], shape[2]), return_sequences=True))
    # output shape: (5, 1)
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [83]:
buildManyToManyModel((0, 5, 5))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_31 (LSTM)               (None, 5, 10)             640       
_________________________________________________________________
time_distributed_10 (TimeDis (None, 5, 1)              11        
Total params: 651
Trainable params: 651
Non-trainable params: 0
_________________________________________________________________


### Train many to many model

In [84]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, pastDay=5, futureDay=5)  # many to one
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, rate=0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)

model = buildOneToManyModel(X_train.shape)
earlystopping = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.001)
csvlog = CSVLogger('many2many_same_train_log')
model.fit(X_train, Y_train, epochs=1000, batch_size=128,
          validation_data=(X_val, Y_val), callbacks=[earlystopping, reducelr, csvlog])

X_train shape (5703, 5, 10)
Y_train shape (5703, 5, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 10)                840       
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 5, 1)              0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5703 samples, validate on 633 samples
Epoch 1/1000
5703/5703 [==============================] - 2s 323us/step - loss: 0.1181 - val_loss: 0.0556
Epoch 2/1000
5703/5703 [==============================] - 0s 33us/step - loss: 0.0288 - val_loss: 0.0082
Epoch 3/1000
5703/5703 [==============================] - 0s 34us/step - loss: 0.0043 - val_los

Epoch 66/1000
5703/5703 [==============================] - 0s 36us/step - loss: 6.9682e-05 - val_loss: 7.5968e-05
Epoch 67/1000
5703/5703 [==============================] - 0s 36us/step - loss: 6.9711e-05 - val_loss: 8.0527e-05
Epoch 68/1000
5703/5703 [==============================] - 0s 35us/step - loss: 6.9103e-05 - val_loss: 7.7010e-05
Epoch 69/1000
5703/5703 [==============================] - 0s 35us/step - loss: 6.8422e-05 - val_loss: 7.6808e-05
Epoch 70/1000
5703/5703 [==============================] - 0s 34us/step - loss: 6.8966e-05 - val_loss: 7.7344e-05
Epoch 71/1000
5703/5703 [==============================] - 0s 34us/step - loss: 6.8528e-05 - val_loss: 7.5882e-05
Epoch 72/1000
5703/5703 [==============================] - 0s 36us/step - loss: 6.8993e-05 - val_loss: 7.6876e-05
Epoch 73/1000
5703/5703 [==============================] - 0s 36us/step - loss: 6.8330e-05 - val_loss: 7.5619e-05
Epoch 74/1000
5703/5703 [==============================] - 0s 35us/step - loss: 6.7541e-

In [85]:
print(X_train[0:1].shape)
print(Y_train[0:1])
#
model.predict(X_train[0:1])

(1, 5, 10)
[[[-0.0799953 ]
  [-0.08803951]
  [-0.08550953]
  [-0.08268756]
  [-0.07934663]]]


array([[[-0.07867057],
        [-0.07867057],
        [-0.07867057],
        [-0.07867057],
        [-0.07867057]]], dtype=float32)

## Many to many with different timesteps

### Construct the model

In [148]:
# Ref: https://github.com/keras-team/keras/issues/6063

def buildManyToManyModel_diff(shape):
    model = Sequential()
    # 10 LSTM node outputs with the last hidden state information
    model.add(LSTM(10, input_shape=(shape[1], shape[2]), return_sequences=False))  
    # transform the tensor as 5 x 10 (timestep x features) format
    model.add(RepeatVector(5))  # time step of future prediction
    model.add(LSTM(10, return_sequences=True))
    model.add(TimeDistributed(Dense(1)))  # apply Dense to every dim and turn the dim to 1 here
    # output shape: (5, 1)
    model.add(Activation('linear'))
    # model.add(BatchNormalization())
    model.compile(loss="mse", optimizer="rmsprop")
    model.summary()
    return model

In [147]:
buildManyToManyModel_diff((0, 30, 10))  # Input的 TimeStep=30, Features=10

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_64 (LSTM)               (None, 10)                840       
_________________________________________________________________
repeat_vector_12 (RepeatVect (None, 5, 10)             0         
_________________________________________________________________
lstm_65 (LSTM)               (None, 5, 5)              320       
_________________________________________________________________
time_distributed_13 (TimeDis (None, 5, 1)              6         
_________________________________________________________________
activation_3 (Activation)    (None, 5, 1)              0         
Total params: 1,166
Trainable params: 1,166
Non-trainable params: 0
_________________________________________________________________


### Train the many to many with different timesteps

In [150]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, pastDay=30, futureDay=5)  # many to one
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, rate=0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

print("X_train shape", X_train.shape)
print("Y_train shape", Y_train.shape)

model = buildManyToManyModel_diff(X_train.shape)
earlystopping = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
csvlog = CSVLogger('many2many_diff_train_log')
# Callbacks要在[]裡
model.fit(X_train, Y_train, epochs=1000, batch_size=128,
          validation_data=(X_val, Y_val), callbacks=[earlystopping, reducelr, csvlog])

X_train shape (5680, 30, 10)
Y_train shape (5680, 5, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_68 (LSTM)               (None, 10)                840       
_________________________________________________________________
repeat_vector_14 (RepeatVect (None, 5, 10)             0         
_________________________________________________________________
lstm_69 (LSTM)               (None, 5, 10)             840       
_________________________________________________________________
time_distributed_15 (TimeDis (None, 5, 1)              11        
_________________________________________________________________
activation_5 (Activation)    (None, 5, 1)              0         
Total params: 1,691
Trainable params: 1,691
Non-trainable params: 0
_________________________________________________________________
Train on 5680 samples, validate on 631 samples
Epoch 1/1000
5680/5680 [=============

5680/5680 [==============================] - 1s 120us/step - loss: 7.1005e-05 - val_loss: 7.6532e-05
Epoch 64/1000
5680/5680 [==============================] - 1s 125us/step - loss: 7.0993e-05 - val_loss: 7.4674e-05
Epoch 65/1000
5680/5680 [==============================] - 1s 125us/step - loss: 7.1048e-05 - val_loss: 7.1700e-05
Epoch 66/1000
5680/5680 [==============================] - 1s 129us/step - loss: 7.0645e-05 - val_loss: 7.1923e-05
Epoch 67/1000
5680/5680 [==============================] - 1s 136us/step - loss: 7.0762e-05 - val_loss: 7.2310e-05
Epoch 68/1000
5680/5680 [==============================] - 1s 133us/step - loss: 7.0667e-05 - val_loss: 7.3629e-05
Epoch 69/1000
5680/5680 [==============================] - 1s 120us/step - loss: 7.0385e-05 - val_loss: 7.8540e-05
Epoch 70/1000
5680/5680 [==============================] - 1s 129us/step - loss: 7.0538e-05 - val_loss: 7.9050e-05
Epoch 71/1000
5680/5680 [==============================] - 1s 117us/step - loss: 7.0459e-05 - 

5680/5680 [==============================] - 1s 131us/step - loss: 6.6024e-05 - val_loss: 6.7596e-05
Epoch 135/1000
5680/5680 [==============================] - 1s 132us/step - loss: 6.5941e-05 - val_loss: 6.8993e-05
Epoch 136/1000
5680/5680 [==============================] - 1s 121us/step - loss: 6.5962e-05 - val_loss: 6.8062e-05
Epoch 137/1000
5680/5680 [==============================] - 1s 125us/step - loss: 6.5364e-05 - val_loss: 6.8712e-05
Epoch 138/1000
5680/5680 [==============================] - 1s 127us/step - loss: 6.5624e-05 - val_loss: 6.9553e-05
Epoch 139/1000
5680/5680 [==============================] - 1s 122us/step - loss: 6.5733e-05 - val_loss: 6.8760e-05
Epoch 140/1000
5680/5680 [==============================] - 1s 137us/step - loss: 6.5547e-05 - val_loss: 6.7365e-05
Epoch 141/1000
5680/5680 [==============================] - 1s 132us/step - loss: 6.5538e-05 - val_loss: 6.7595e-05
Epoch 142/1000
5680/5680 [==============================] - 1s 135us/step - loss: 6.561

Epoch 205/1000
5680/5680 [==============================] - 1s 140us/step - loss: 6.3129e-05 - val_loss: 6.6194e-05
Epoch 206/1000
5680/5680 [==============================] - 1s 142us/step - loss: 6.3205e-05 - val_loss: 6.6485e-05
Epoch 207/1000
5680/5680 [==============================] - 1s 135us/step - loss: 6.3535e-05 - val_loss: 6.4482e-05
Epoch 208/1000
5680/5680 [==============================] - 1s 135us/step - loss: 6.2843e-05 - val_loss: 6.6100e-05
Epoch 209/1000
5680/5680 [==============================] - 1s 133us/step - loss: 6.3222e-05 - val_loss: 6.6521e-05
Epoch 210/1000
5680/5680 [==============================] - 1s 131us/step - loss: 6.3071e-05 - val_loss: 6.5367e-05
Epoch 211/1000
5680/5680 [==============================] - 1s 127us/step - loss: 6.2892e-05 - val_loss: 6.4964e-05
Epoch 212/1000
5680/5680 [==============================] - 1s 134us/step - loss: 6.3203e-05 - val_loss: 6.5086e-05
Epoch 213/1000
5680/5680 [==============================] - 1s 142us/ste

In [152]:
print(X_train[0:1].shape)
print(Y_train[0:4].shape)
print(Y_train[0:4])
#
model.predict(X_train[0:4])

(1, 30, 10)
(4, 5, 1)
[[[-0.00594545]
  [-0.00457002]
  [-0.00938424]
  [-0.01333908]
  [-0.00800885]]

 [[-0.13893596]
  [-0.13621379]
  [-0.12705495]
  [-0.1361498 ]
  [-0.14165801]]

 [[ 0.01471112]
  [ 0.01632962]
  [ 0.01724575]
  [ 0.01501666]
  [ 0.01519975]]

 [[-0.04980785]
  [-0.03762988]
  [-0.03094039]
  [-0.03488536]
  [-0.03754408]]]


array([[[-0.01961062],
        [-0.0211651 ],
        [-0.02226461],
        [-0.02317771],
        [-0.02379746]],

       [[-0.13567643],
        [-0.14068054],
        [-0.14062285],
        [-0.14021252],
        [-0.13992606]],

       [[ 0.01421083],
        [ 0.01309283],
        [ 0.01238822],
        [ 0.01206255],
        [ 0.01206986]],

       [[-0.05316778],
        [-0.05156799],
        [-0.05114401],
        [-0.0517293 ],
        [-0.05263855]]], dtype=float32)